## The required format for the GLAD program is:
```
First line:
    <numGivenLabels> <numLabelers> <numTasks> <Prior p(Z=1)>
  Following <numGivenLabels> lines:
    <taskId> <labelerId> <label:0|1>
```
0: Not a Disease (Includes "Phenotype")
<br>
1: Disease (Includes "Both")

I think task refers to the word in the dataset, and then labeler ID and the label. So if we have 20000 words in M2C and 5 label, the output of this program should have 100000 lines.

In [1]:
dataset_folder = '../dataset/Mark2Cure_NER_dataset/'

In [2]:
citizens = ['1', '2', '3', '4', '5']

In [3]:
filename = 'M2C{}-Disease_Phen_Preliminary_Run_1st_pass.txt'

In [4]:
text = ''

In [5]:
## Make vocab
vocab = {}
reverse_vocab = {}
count_vocab = dict()
path = dataset_folder + filename.format('1')
i = 0
total_lines = 0
with open(path) as f:
    lines = f.read().split('\n')
    for line in lines:
        if line:
            total_lines += 1
            word = line.split()[0]
            word = word.lower()
            c = count_vocab.get(word, 0)
            count_vocab[word] = c + 1
            word = word + '_{}'.format(i)
            if word.lower() not in vocab:
                vocab[word.lower()] = i
                reverse_vocab[i] = word.lower()
                i += 1

In [6]:
len(vocab), len(count_vocab)

(7863, 2245)

In [17]:
# make 5 arrays of text
annotations = []
for citizen in citizens:
    path = dataset_folder + filename.format(citizen)
    with open(path) as f:
        lines = f.read().split('\n')
        annotations.append(lines)

In [18]:
text = '{} {} {} {} {} {}\n'.format(len(vocab)*len(annotations), len(annotations), len(vocab), 2, 0.5, 0.5)

In [19]:
len(annotations), total_lines

(5, 7863)

In [20]:
record = dict()
for line in range(total_lines):
    for annotator in range(len(annotations)):
        current_line = annotations[annotator][line].split()
        word = current_line[0].lower() + '_{}'.format(line)
        annotation = current_line[3]
        count = record.get(word, 0)
        if count < len(annotations):
            label = '1'
            if annotation == 'O':
                label = '0'
            text = text + '{} {} {}\n'.format(vocab[word], annotator, label)
            record[word] = count + 1

In [21]:
with open('data_combined_wordidx.txt', 'w') as f:
    f.write(text)

## The following code is to convert the alpha, beta and label csv to readable formats AFTER we run GLAD

### Make sure you have the proper files in the proper folder before running these hacky snippets

In [23]:
results_folder = '../Results/Results for word_idx + Combined Phenotypes and Disease/'

In [24]:
dict_betas = {}
text = ''
with open(results_folder+'beta.csv', 'r') as f:
    lines = f.readlines()
    for line in lines:
        if line:
            arr = line.split(',')
            num = arr[0]
            if num == '# id':
                continue
            word = reverse_vocab[int(num)-1]
            beta_temp = float(arr[1])
            text += '{}\t{}\n'.format(word, beta_temp)
with open(results_folder+'beta_word_idx_combined_independant.csv','w') as g:
    g.write(text)

In [17]:
text = ''
with open(results_folder+'label.csv', 'r') as f:
    lines = f.readlines()
    for line in lines:
        if line:
            arr = line.split(',')
            num = arr[0]
            if num == '# id':
                continue
            word = reverse_vocab[int(num)-1]
            
            p_0 = float(arr[1])
            p_1 = float(arr[2])
            
            text += '{}\t{}\t{}\n'.format(word, p_0, p_1)
    
with open(results_folder+'label_word_idx_independant.csv','w') as g:
    g.write(text)